# Robot controller using Jupyros

## Importing library

In [2]:
import os
import numpy as np
import rospy
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from move_base_msgs.msg import MoveBaseActionGoal
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import HBox
from actionlib_msgs.msg import GoalID, GoalStatusArray
from IPython.display import display, clear_output
from ipywidgets import interactive_output, FloatText, Button, Checkbox, Output, ToggleButtons

## Main function

### Initializing, definition in a class named RobotController 

In [12]:
class RobotController():
    
    
    def __init__(self):
        rospy.init_node("robot_controller")
        self.vel_publish = rospy.Publisher("/cmd_vel", Twist, queue_size=1)
        self.movebase_publish = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size=1)
        self.cancell_publish = rospy.Publisher("/move_base/cancel", GoalID, queue_size=1)
        self.lin = 0.0
        self.ang = 0.0
        self.assisted = False
        self.reached = 0
        self.canceled = 0
        self.status = -1
        self.flag=False
        self.wall_th = 1  # Define the wall threshold value here
        self.sub_status = rospy.Subscriber("/move_base/status", GoalStatusArray, self.status_callback)
        self.sub_scan = rospy.Subscriber("/scan", LaserScan, self.assisted_drive)
        self.sub_cmd_vel = rospy.Subscriber("/cmd_vel", Twist, self.cmd_vel_callback)
        # Create the driving mode toggle buttons
        self.driving_mode = ToggleButtons(
            options=[("Reach point", 1), ("Drive manually", 2)],
            description="Select Mode:",
            value=1,
            disabled=False,
            button_style='primary', 
            style={'button_width': 'auto', 'font-weight': 'bold'}  
        )

        self.output1 = Output()
        self.output2 = Output()
        self.display_widgets()

        
    def display_widgets(self):
        display(self.driving_mode)
        display(self.output1)
        display(self.output2)
        # Bind the driving mode selection to the corresponding function
        interactive_output(self.choose_mode, {'mode': self.driving_mode})
        

    def choose_mode(self, mode):
        if mode == 1:
            self.reach_point()
        elif mode == 2:
            self.drive_manually()
            

    def status_callback(self, msg):
        if self.flag:
            goal_id = GoalID()
            self.status = msg.status_list[0].status

            if self.status == 3:
                print("goal reached!")
                self.status = -1
                self.cancell_publish.publish(goal_id)
                self.reached += 1
                self.flag = False

            if self.status == 4:
                print("The goal cancelled!")
                self.status = -1
                self.cancell_publish.publish(goal_id)
                self.canceled += 1
                self.flag = False

            if self.status == 5:
                print("The goal was rejected!")
                self.status = -1
                self.cancell_publish.publish(goal_id)
                self.canceled += 1
                self.flag = False

                
    def assisted_drive(self, msg):
        if self.assisted:
            vel = Twist()
            # Distance of the closest obstacle on the right, front, left of the robot
            right = min(min(msg.ranges[0:69]), 1)
            front = min(min(msg.ranges[300:419]), 1)
            left = min(min(msg.ranges[650:719]), 1)

            vel.linear.x = self.lin
            vel.angular.z = self.ang
            if front < self.wall_th:
                vel.linear.x = 0
            elif left < self.wall_th:
                vel.angular.z = -1
            elif right < self.wall_th:
                vel.angular.z = 1
            else:
                pass

            self.vel_publish.publish(vel)
            

    def cmd_vel_callback(self, msg):
        self.lin = msg.linear.x
        self.ang = msg.angular.z

        
    def reach_point(self):   # Function to handle reaching a point autonomously using movebase
        with self.output1:
            clear_output(wait=True)
            set_x = FloatText(description="X:",  disabled=False, style={'description_width': 'initial'}, layout={'width': '10%'})
            set_y = FloatText(description="Y:",  disabled=False, style={'description_width': 'initial'}, layout={'width': '10%'})
            button_set = Button(description="Send goal", button_style='success', style={'button_color': 'green'})
            button_canc = Button(description="Cancel goal", button_style='danger', style={'button_color': 'red'})
            output_set = Output()
            output_canc = Output()

            display(HBox([set_x, set_y]), button_set, output_set)
            display(button_canc, output_canc)

            button_set.on_click(lambda b: self.set_goal(b, set_x.value, set_y.value))
            button_canc.on_click(self.cancel_goal)
            
            
    def set_goal(self, b, x_coord, y_coord):
        with self.output1:
            clear_output(wait=True)
            print("Current goal: (%f, %f)" % (x_coord, y_coord))

            my_goal = MoveBaseActionGoal()
            my_goal.goal.target_pose.pose.position.x = x_coord
            my_goal.goal.target_pose.pose.position.y = y_coord
            my_goal.goal.target_pose.header.frame_id = "map"
            my_goal.goal.target_pose.pose.orientation.w = 1

            self.movebase_publish.publish(my_goal)
            self.flag = True
            
   
    def cancel_goal(self, b):   # Function to handle canceling the current goal
        with self.output1:
            clear_output(wait=True)
            if self.flag:
                id_cancel = GoalID()
                self.cancell_publish.publish(id_cancel)
                print("goal cancelled!")
                self.canceled += 1
            else:
                print("No goal set!")     
                
                
            
    def drive_manually(self):   # Function to handle driving the robot manually
        with self.output2:
            clear_output(wait=True)
            styles={'button_color': 'darkslateblue', 'font_weight': 'bold'}
            self.assisted_drive_checkbox = Checkbox(value=False, description="Assisted drive", disabled=False, indent=False)
            button_front = Button(description="Go forward",style=styles)
            button_right = Button(description="Turn right",style=styles)
            button_back = Button(description="Go backward",style=styles)
            button_left = Button(description="Turn left",style=styles)
            button_increase = Button(description="Increase velocity",style=styles)
            button_decrease = Button(description="Decrease velocity",style=styles)
            button_stop = Button(description="Stop robot",style=styles)

            row1 = HBox([button_front, button_back])
            row2 = HBox([button_left, button_right])
            row3 = HBox([button_increase, button_decrease, button_stop])

            display(
                self.assisted_drive_checkbox,
                row1,
                row2,
                row3
            )
            self.assisted_drive_checkbox.observe(self.set_assisted_drive)
            button_front.on_click(self.move_forward)
            button_right.on_click(self.turn_right)
            button_back.on_click(self.move_backward)
            button_left.on_click(self.turn_left)
            button_increase.on_click(self.increase_velocity)
            button_decrease.on_click(self.decrease_velocity)
            button_stop.on_click(self.stop_robot)

            
            
    def set_assisted_drive(self, change):
        self.assisted = change.new

    def move_forward(self, b): # Function to handle moving the robot forward
        vel = Twist()
        vel.linear.x = 0.5
        vel.angular.z = 0
        self.vel_publish.publish(vel)

    def turn_right(self, b):
        vel = Twist()
        vel.linear.x = 0
        vel.angular.z = -1
        self.vel_publish.publish(vel)

    def move_backward(self, b):# Function to handle moving the robot backward
        vel = Twist()
        vel.linear.x = -0.5
        vel.angular.z = 0
        self.vel_publish.publish(vel)

    def turn_left(self, b):
        vel = Twist()
        vel.linear.x = 0
        vel.angular.z = 1
        self.vel_publish.publish(vel)

    def increase_velocity(self, b):
        vel = Twist()
        self.lin = self.lin *1.5
        self.ang = self.ang *1.5
        self.vel_publish.publish(vel)

    def decrease_velocity(self, b):  
        vel = Twist()
        self.lin = self.lin *0.5
        self.ang = self.ang *0.5
        self.vel_publish.publish(vel)

    def stop_robot(self, b): # Function to handle stopping the robot
        vel = Twist()
        vel.linear.x = 0
        vel.angular.z = 0
        self.vel_publish.publish(vel)



    def show_goals(self, b):
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.bar(['Reached', 'Canceled'], [self.reached, self.canceled], align='center', alpha=0.6, color=['g', 'r'])
        plt.title('Goals History')
        plt.xlabel('Goals')
        plt.ylabel('Count')
        plt.show()

## Run the code

In [13]:
robot_controller = RobotController() ## Create an instance of the RobotController class



ToggleButtons(button_style='primary', description='Select Mode:', options=(('Reach point', 1), ('Drive manuall…

Output()

Output()